In [1]:
import zipfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
# sphinx_gallery_thumbnail_path = '_static/playback-thumbnail.png'

import librosa
# We'll need IPython.display's Audio widget
from IPython.display import Audio

# We'll also use `mir_eval` to synthesize a signal for us
import mir_eval.sonify

import csv
import os
import json
from pydub import AudioSegment



#TODO: extract only comp audio files

e:\MDNE\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def unzip_file(data_path, zipped_name):
    '''
    Descomprime arquivo específico dentro da pasta aux_folder

    Args
    data_path (str) : localização do arquivo .zip
    zipped_name (str): localização do arquivo dentro do .zip


    '''
    with zipfile.ZipFile(data_path) as zf:
        zf.extract(zipped_name, 'aux_folder/')
        #zf.extract('dir_sub/file_sub.txt', 'dir_out_extract')


def name_jams(audio_name):
    '''
    Recupera o nome do arquivo de metadados dentro de annotation.zip
    '''
    jams_name = audio_name.replace('_hex', '').replace('wav', 'jams')
    return jams_name


def remove_solos(files_list):
    '''
    Remove arquivos com solo no nome de uma lista

    Args:
    files_list (list): lista contendo o nome de todos os arquivos da base de dados

    '''
    res = []
    for i in files_list:
        if 'solo' in i:
            pass
        else:
            res.append(i)
    
    return res




In [10]:
def create_folder():
    ''' 
    Cria as pasta auxiliares na raiz do projeto
    '''
    os.mkdir('aux_folder')
    os.mkdir('processed_audio')
    


def read_metadata(metadata_name):
    ''' 
    Pega o nome do acorde no arquivo de metadados 

    Args:
    metadata_name (str): nome do metadado (arquivo .json)
    '''
    # Opening JSON file
    f = open('aux_folder/' + metadata_name)
    
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    chord = data['annotations'][14]['data']
    #key_mode = data['annotations'][16]['data']

    return chord




#ref -> https://thepythoncode.com/assistant/transformation-details/cutting-audio-files-in-python-with-pydub/
from pydub import AudioSegment

def cut_audio(input_file, output_file, start_time, end_time):
    """
    Cut parts of an audio file and save it to a new file.
    Corta as parte do aúdio e separa em arquivo diferentes
    

    Args:
        input_file (str): caminho para o arquivo que você quer cortar
        output_file (str): caminho onde salvar os cortes 
        start_time (int): tempo onde você quer começar o corte em ms
        end_time (int): tempo onde você quer terminar o corte em ms
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Cut the audio
    cut_audio = audio[start_time:end_time]

    # Save the cut audio to a new file
    cut_audio.export(output_file, format="wav")





def cut_all(chord_dict, audio_path, output_path, og_file, batch):
    '''
    Corta todos os aúdios .jams em arquivo independentes para cada acorde

    chord_dict: dict, contendo todos os acordes do áudio original
    audio_path: str, caminho para áudio original
    output_path: str, pasta onde os arquivo cortados serão salvos
    og_file: str, nome do arquivo original para documentação
    '''
    iter = np.random.randint(0,999)
    for i in chord_dict:

        start = i['time']*1000
        end = (i['time'] + i['duration'])*1000
        value = i['value']
        filename = ('file_' + str(batch) + str(iter) + '.wav').replace(':', '')
        output = (output_path + filename)
        
        cut_audio(audio_path, output, start, end)
        # Documenta metadados do arquivo cortado
        with open('metadata_v3.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([filename, output_path, value, og_file])
        
        iter +=1




In [11]:
from tqdm import tqdm
def preprocess_audio():
    ''' 
    Pipeline completa de preprocessamento.
    Separa os aúdios em aúdio independentes para cada acorde e salva metadados
    '''

    # zip file handler  
    data_path = 'audio_hex-pickup_original.zip'
    annotations_path = 'annotation.zip'

    create_folder()

    aux_folder = 'aux_folder/'
    prepro_folder = 'processed_audio/'


    zip = zipfile.ZipFile(data_path)
    # arquivos disponíveis no arquivo zipado
    zipped_files = zip.namelist()

    # remove versões de solo 
    zipped_files = remove_solos(zipped_files)
    total_chords = 0
    batch = 1
    for i in tqdm(zipped_files):
        metadata_name = name_jams(i)
        

        # Unzipa o arquivo i dentro da pasta aux_folder
        unzip_file(data_path, i)
        # Unzipa o arquivo .jams de metadados do arquivo i dentro da pasta aux_folder
        unzip_file(annotations_path, metadata_name)

        # Recupera o dicionário que marca a duração de cada acorde
        chord = read_metadata(metadata_name)
        
        
        cut_all(chord, aux_folder + i, prepro_folder, i, batch)

        total_chords += len(chord)
        os.remove(aux_folder + i)
        os.remove(aux_folder + metadata_name)

        batch +=1
    
    return f'{total_chords} chords were (hopefully!) extracted'

In [12]:
preprocess_audio()

100%|██████████| 180/180 [01:15<00:00,  2.39it/s]


'2160 chords were (hopefully!) extracted'